In [ ]:
import random
from datetime import datetime

import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import StratifiedKFold
from torch import nn, optim
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from tqdm import tqdm

In [ ]:
from sklearn.model_selection import train_test_split

#raw_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/기계학습/data/train.csv')
raw_data = pd.read_csv('train.csv')

raw_data.loc[raw_data['voted']==2, 'voted']=0
Answers = ['QaA', 'QbA', 'QcA', 'QdA', 'QeA',
             'QfA', 'QgA', 'QhA', 'QiA', 'QjA', 
             'QkA', 'QlA', 'QmA', 'QnA', 'QoA', 
             'QpA', 'QqA', 'QrA', 'QsA', 'QtA']

flipping_columns = ["QeA", "QfA", "QkA", "QqA", "QrA"]
for flip in flipping_columns: 
    raw_data[flip] = 6 - raw_data[flip]

flipping_secret_columns = ["QaA", "QdA", "QgA", "QiA", "QnA"]
for flip in flipping_secret_columns: 
    raw_data[flip] = 6 - raw_data[flip]

raw_data['Mach_score'] = raw_data[Answers].sum(axis = 1)
raw_data.drop(['index'], axis=1, inplace=True)
data = raw_data.copy()

In [ ]:
index1 = data.loc[data['familysize'] >= 50].index
data1 = data.drop(index1, axis=0)   #가족 수 50명이상 삭제

data2 = data.copy()
data2.loc[data2['familysize'] >= 5, 'familysize'] = 5    #5명이상은 5로 퉁

data3 = data.copy()
data3.loc[data3['familysize'] >= 10, 'familysize'] = 10  #10명이상은 10으로 퉁

index2 = data.loc[data['familysize'] >= 10].index
data4 = data.drop(index2, axis=0)   #가족 수 10명이상 삭제

index3 = data.loc[data['familysize'] >= 6].index
data5 = data.drop(index3, axis=0)   #가족 수 6명이상 삭제

#data_list = [data, data1, data2, data3, data4, data5]
data = data5
data.shape

(43795, 78)

In [ ]:
one_hot1 = pd.get_dummies(data['age_group'])
one_hot2 = pd.get_dummies(data['race'])
one_hot3 = pd.get_dummies(data['religion'])
one_hot4 = pd.get_dummies(data['familysize'])


data = pd.concat([data, one_hot1, one_hot2, one_hot3, one_hot4], axis=1)
data.drop(['age_group', 'race', 'religion', 'familysize'], axis=1, inplace=True)
data

,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,QeE,...,Jewish,Muslim,Other,Sikh,0,1,2,3,4,5
0,3.0,363,4.0,1370,5.0,997,5.0,1024,4.0,1577,...,0,0,1,0,0,0,0,0,1,0
1,1.0,647,5.0,1313,3.0,3387,1.0,2969,5.0,4320,...,0,0,0,0,0,0,0,1,0,0
2,2.0,1623,1.0,1480,1.0,1021,2.0,3374,1.0,1333,...,0,0,1,0,0,0,0,1,0,0
3,3.0,504,3.0,2311,4.0,992,3.0,3245,5.0,357,...,0,0,0,0,1,0,0,0,0,0
4,5.0,927,1.0,707,5.0,556,4.0,1062,5.0,1014,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45527,4.0,1050,5.0,619,4.0,328,5.0,285,5.0,602,...,1,0,0,0,0,1,0,0,0,0
45528,4.0,581,3.0,1353,4.0,1164,5.0,798,3.0,1680,...,0,0,0,0,0,0,1,0,0,0
45529,2.0,593,1.0,857,1.0,1047,2.0,1515,1.0,1690,...,0,0,0,0,0,1,0,0,0,0
45530,5.0,747,3.0,1331,4.0,892,4.0,1281,5.0,1328,...,0,0,0,0,0,0,1,0,0,0


In [ ]:



from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
data['gender'] = label_encoder.fit_transform(data['gender'])

train = data.drop('voted', axis=1)
drop_list = ['QaE', 'QbE', 'QcE', 'QdE', 'QeE',
             'QfE', 'QgE', 'QhE', 'QiE', 'QjE',
             'QkE', 'QlE', 'QmE', 'QnE', 'QoE',
             'QpE', 'QqE', 'QrE', 'QsE', 'QtE']
d2 = ['QaA', 'QbA', 'QcA', 'QdA', 'QeA', 'QfA', 'QgA', 'QhA', 'QiA', 'QjA',
       'QkA', 'QlA', 'QmA', 'QnA', 'QoA', 'QpA', 'QqA', 'QrA', 'QsA', 'QtA']

train = train.drop(drop_list, axis=1)
train = train.drop(d2, axis=1)


target = data['voted']
#train.to_csv('10명이상은10_모두타겟인코딩.csv', index=False)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(train, target, test_size=0.20, random_state=42, stratify=target)
print(x_train.shape, x_test.shape)

(35036, 65) (8759, 65)


In [ ]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

train_y = y_train.to_numpy()
train_x = x_train.to_numpy()
test_x = x_test.to_numpy()

train_y_t = torch.tensor(train_y, dtype=torch.float32)
train_x_t = torch.tensor(train_x, dtype=torch.float32)
test_x_t = torch.tensor(test_x, dtype=torch.float32)
test_len = len(test_x_t)

N_REPEAT = 1
N_SKFOLD = 5
N_EPOCH = 200
BATCH_SIZE = 64
LOADER_PARAM = {
    'batch_size': BATCH_SIZE,
    'num_workers': 2,
    'pin_memory': True
}
prediction = np.zeros((test_len, 1), dtype=np.float32)

for repeat in range(N_REPEAT):

    skf, tot = StratifiedKFold(n_splits=N_SKFOLD, random_state=repeat, shuffle=True), 0.
    for skfold, (train_idx, valid_idx) in enumerate(skf.split(train_x, train_y)):
        train_idx, valid_idx = list(train_idx), list(valid_idx)
        train_loader = DataLoader(TensorDataset(train_x_t[train_idx, :], train_y_t[train_idx]),
                                  shuffle=True, drop_last=True, **LOADER_PARAM)
        valid_loader = DataLoader(TensorDataset(train_x_t[valid_idx, :], train_y_t[valid_idx]),
                                  shuffle=False, drop_last=False, **LOADER_PARAM)
        test_loader = DataLoader(TensorDataset(test_x_t, torch.zeros((test_len,), dtype=torch.float32)),
                                 shuffle=False, drop_last=False, **LOADER_PARAM)
        model = nn.Sequential(
            nn.Dropout(0.05),
            nn.Linear(65, 180, bias=False),
            nn.LeakyReLU(0.05, inplace=True),
            nn.Dropout(0.5),
            nn.Linear(180, 32, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(32, 1)
        ).to(DEVICE)
        criterion = torch.nn.BCEWithLogitsLoss(pos_weight=torch.tensor([1.20665], device=DEVICE))
        optimizer = optim.AdamW(model.parameters(), lr=5e-3, weight_decay=7.8e-2)
        scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
            optimizer, T_0=N_EPOCH // 6, eta_min=4e-4)
        prediction_t, loss_t = np.zeros((test_len, 1), dtype=np.float32), 1.

        # for epoch in range(N_EPOCH):
        for epoch in tqdm(range(N_EPOCH), desc='{:02d}/{:02d}'.format(skfold + 1, N_SKFOLD)):
            model.train()
            for idx, (xx, yy) in enumerate(train_loader):
                optimizer.zero_grad()
                xx, yy = xx.to(DEVICE), yy.to(DEVICE)
                pred = model(xx).squeeze()
                loss = criterion(pred, yy)
                loss.backward()
                optimizer.step()
                scheduler.step(epoch + idx / len(train_loader))

            with torch.no_grad():
                model.eval()
                running_acc, running_loss, running_count = 0, 0., 0
                for xx, yy in valid_loader:
                    xx, yy = xx.to(DEVICE), yy.to(DEVICE)
                    pred = model(xx).squeeze()
                    loss = criterion(pred, yy)
                    running_loss += loss.item() * len(yy)
                    running_count += len(yy)
                    running_acc += ((torch.sigmoid(pred) > 0.5).float() == yy).sum().item()
                print('R{:02d} S{:02d} E{:02d} | {:6.4f}, {:5.2f}%'
                       .format(repeat + 1, skfold + 1, epoch + 1, running_loss / running_count,
                               running_acc / running_count * 100))

                if running_loss / running_count < loss_t:
                    loss_t = running_loss / running_count
                    for idx, (xx, _) in enumerate(test_loader):
                        xx = xx.to(DEVICE)
                        pred = (2. - torch.sigmoid(model(xx).detach().to('cpu'))).numpy()
                        prediction_t[BATCH_SIZE * idx:min(BATCH_SIZE * (idx + 1), len(prediction)), :] \
                            = pred[:, :].copy()
        prediction[:, :] += prediction_t[:, :].copy() / (N_REPEAT * N_SKFOLD)
        tot += loss_t
    print('R{} -> {:6.4f}'.format(repeat + 1, tot / N_SKFOLD))

01/05:   0%|          | 0/300 [00:00<?, ?it/s]

R01 S01 E01 | 0.6469, 66.99%


01/05:   0%|          | 1/300 [00:02<13:34,  2.72s/it]

R01 S01 E02 | 0.6412, 67.48%


01/05:   1%|          | 3/300 [00:07<11:59,  2.42s/it]

...

R01 S05 E298 | 0.6165, 69.40%


05/05: 100%|█████████▉| 299/300 [11:55<00:02,  2.34s/it]

R01 S05 E299 | 0.6164, 69.57%


05/05: 100%|██████████| 300/300 [11:57<00:00,  2.39s/it]

R01 S05 E300 | 0.6169, 69.27%
R1 -> 0.6214
